In [4]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Literal, Annotated
from langchain_google_vertexai import ChatVertexAI
from langchain_core.messages import SystemMessage, HumanMessage
import operator

In [5]:
generator_llm=ChatVertexAI(model_name="gemini-2.5-pro")
evaluator_llm=ChatVertexAI(model_name="gemini-2.5-flash")
optimizer_llm=ChatVertexAI(model_name="gemini-2.5-pro")

In [6]:
from pydantic import BaseModel, Field
class TweetEvaluation(BaseModel):
    evaluation:Literal["approved","needs_improvement"]=Field(...,description="Final evaluation result")
    feedback:str=Field(...,description="feedback for the tweet.")

In [7]:
structured_evaluator_llm=evaluator_llm.with_structured_output(TweetEvaluation)

In [8]:
class TweetState(TypedDict):
    topic:str
    tweet:str
    evaluation:Literal["approved","needs_improvement"]
    feedback:str
    iteration:int
    max_iteration:int

    tweet_history:Annotated[list[str],operator.add]
    feedback_history:Annotated[list[str],operator.add]

In [9]:
def generate_tweet(state:TweetState):
    messages=[
        SystemMessage(content=""),
        HumanMessage(content="")
    ]
    response=generator_llm.invoke(messages).content
    return {'tweet':response,'tweet_history':[response]}

In [10]:
def evaluate_tweet(state:TweetState):
    messages=[
        SystemMessage(content=""),
        HumanMessage(content="")
    ]
    response=structured_evaluator_llm.invoke(messages)
    return {'evaluation':response.evaluation,
            'feedback':response.feedback,'feedback_history':[response.feedback]}

In [12]:
def optimize_tweet(state:TweetState):
    messages=[
        SystemMessage(content=""),
        HumanMessage(content="")
    ]
    response=optimizer_llm.invoke(messages).content
    iteration=state['iteration']+1
    return {'tweet':response,'iteration':iteration,'tweet_history':[response]}

In [13]:
def route_evaluation(state:TweetState):
    if state['evaluation']=='approved' or state['iteration']>=state['max_iteration']:
        return 'approved'
    else:
        return 'needs_improvement'